<a href="https://colab.research.google.com/github/MinuMaryS/Coursera_Capstone/blob/main/CourseraCapstone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np


<h3>This is my final capstone project document. 

The aim of the project is to analyze neighbourhoods in Toronto.


In [2]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!
